In [2]:
import os
import json
from typing import Dict, List, Any, Optional
from datetime import datetime
from dotenv import load_dotenv
import requests
from groq import Groq

# Load environment variables
load_dotenv()

# Initialize Groq client
groq_api_key = os.getenv("GROQ_API_KEY")
if not groq_api_key:
    raise ValueError("GROQ_API_KEY not found in environment variables")

groq_client = Groq(api_key=groq_api_key)

class JobRecommendationAgent:
    def __init__(self, user_data: Dict[str, Any]):
        """Initialize the agent with user profile data."""
        self.user_data = user_data
        self.skills = user_data.get('skills', [])
        self.education = user_data.get('education', '')
        self.experience_years = user_data.get('experience_years', {}).get('$numberInt', 0)
        self.last_job = user_data.get('last_job', {})
        self.job_preferences = user_data.get('job_preferences', {})
        self.location = user_data.get('location', {})
        self.community_preferences = user_data.get('community', {})
        
    def _extract_profile_summary(self) -> str:
        """Create a summary of the user profile for the AI prompt."""
        last_job_title = self.last_job.get('title', 'None')
        last_job_company = self.last_job.get('company', 'None')
        
        return f"""
        Education: {self.education}
        Skills: {', '.join(self.skills)}
        Experience: {self.experience_years} years
        Last Job: {last_job_title} at {last_job_company}
        Job Preferences: {self.job_preferences.get('type', 'Any')}
        Preferred Roles: {', '.join(self.job_preferences.get('roles', ['Any']))}
        Location: {self.location.get('city', 'Not specified')}
        Willing to Relocate: {self.location.get('relocation', False)}
        Work Mode Preference: {self.location.get('work_mode', 'Not specified')}
        Short-term Goal: {self.job_preferences.get('short_term_goal', 'Not specified')}
        Long-term Goal: {self.job_preferences.get('long_term_goal', 'Not specified')}
        Wants Mentorship: {self.community_preferences.get('wants_mentorship', False)}
        Mentorship Type: {self.community_preferences.get('mentorship_type', 'Not specified')}
        Interested in Events: {self.community_preferences.get('join_events', False)}
        """
    
    def get_response(self, user_query: str) -> str:
        """Generate a response based on the user query using Groq API."""
        profile_summary = self._extract_profile_summary()
        
        # Determine query type
        query_type = self._determine_query_type(user_query)
        
        # Create prompt based on query type
        prompt = self._create_prompt(user_query, profile_summary, query_type)
        
        # Get response from Groq API
        try:
            response = groq_client.chat.completions.create(
                model="llama3-70b-8192",  # Using llama3 model
                messages=[
                    {"role": "system", "content": "You are a career advisor and job recommendation AI assistant. Your goal is to provide personalized job recommendations, community groups, and events based on the user's profile and query."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.5,
                max_tokens=2000
            )
            
            return response.choices[0].message.content
        except Exception as e:
            return f"Error getting recommendations: {str(e)}"
    
    def _determine_query_type(self, user_query: str) -> str:
        """Determine what type of recommendations the user is looking for."""
        user_query = user_query.lower()
        
        if any(keyword in user_query for keyword in ['job', 'career', 'work', 'employment', 'position', 'opportunity', 'hiring']):
            return 'jobs'
        elif any(keyword in user_query for keyword in ['community', 'group', 'network', 'connect', 'people', 'professional']):
            return 'groups'
        elif any(keyword in user_query for keyword in ['event', 'webinar', 'workshop', 'conference', 'meetup', 'upskill', 'learn']):
            return 'events'
        else:
            return 'general'
    
    def _create_prompt(self, user_query: str, profile_summary: str, query_type: str) -> str:
        """Create a detailed prompt for the AI based on query type."""
        base_prompt = f"""
        USER PROFILE:
        {profile_summary}
        
        USER QUERY: {user_query}
        
        """
        
        if query_type == 'jobs':
            base_prompt += """
            Based on this user's profile and query, recommend 5 relevant job opportunities. For each job:
            1. Include the job title, company, and location
            2. Explain why this job is a good match for their skills and experience
            3. Provide a link to apply (you can suggest links from LinkedIn, Internshala, Naukri, or company career pages)
            4. Format your response in a clear, structured way
            
            Focus on jobs that match their skills: Python, Java, AI, NLP, ML, DL, web development, and app development.
            Consider their experience level, preferred location, and work mode preferences.
            """
        elif query_type == 'groups':
            base_prompt += """
            Based on this user's profile and query, recommend 5 relevant professional groups or communities to join. For each group:
            1. Include the group name and platform (e.g., LinkedIn, Discord, Slack)
            2. Explain why this group would be beneficial for their career growth
            3. Provide approximate member count if possible
            4. Suggest how to use this group effectively
            
            Focus on groups related to their skills: Python, Java, AI, NLP, ML, DL, web development, and app development.
            Consider both online and local groups if applicable to their location.
            """
        elif query_type == 'events':
            base_prompt += """
            Based on this user's profile and query, recommend 5 relevant upcoming events, webinars, or workshops. For each event:
            1. Include the event name, organizer, date, and whether it's online or in-person
            2. Explain why this event is relevant to their career goals and skills
            3. Include registration information or links
            4. Mention any costs involved or if it's free
            
            Focus on events related to: Python, Java, AI, NLP, ML, DL, web development, and app development.
            Consider their location and preference for skill development.
            """
        else:
            base_prompt += """
            Based on this user's profile and query, provide a helpful response that might include a mix of:
            1. Job recommendations
            2. Community groups to join
            3. Upcoming events or learning opportunities
            4. Career advice specific to their situation
            
            Focus on their skills: Python, Java, AI, NLP, ML, DL, web development, and app development.
            Consider their location, experience level, and career goals.
            """
        
        base_prompt += """
        Ensure your recommendations are specific, actionable, and personalized to this user's profile.
        Format your response in a clear, organized manner with appropriate headings and spacing.
        """
        
        return base_prompt

# Example implementation
def main():
    # Sample user data (in practice, this would be fetched from MongoDB)
    sample_user_data = {
        "_id": {"$oid": "6809c3daa03a7a1e240ab91f"},
        "user_id": "6809c002a03a7a1e240ab91e",
        "education": "Bachelor's Degree",
        "skills": ["Python", "Java", "AI", "NLP", "ML", "DL", "web development", "app development"],
        "current_status": "Looking for Work",
        "experience_years": {"$numberInt": "1"},
        "last_job": {"title": "AI developer", "company": "OLVT"},
        "life_stage": {"pregnancy_status": "No", "needs_flexible_work": False, "situation": "None of the above"},
        "job_preferences": {
            "type": "Remote Work",
            "roles": ["Software"],
            "short_term_goal": "Upskill and crack good placement",
            "long_term_goal": "Yes, i want to be an entrepreneur"
        },
        "location": {"city": "Tirupati", "relocation": True, "work_mode": "Flexible"},
        "community": {"wants_mentorship": True, "mentorship_type": "Skill development", "join_events": True},
        "communication_preference": "Email",
        "consent": True,
        "created_at": {"$date": {"$numberLong": "1745470426051"}}
    }
    
    # Initialize the agent
    agent = JobRecommendationAgent(sample_user_data)
    
    # Example queries to test
    example_queries = [
        "Can you recommend jobs for me?",
        "What tech communities should I join for AI and ML?",
        "Are there any upcoming webinars for Python and AI skills?",
        "What are my career options with my current skills?"
    ]
    
    # Test the agent with the first query
    print("\n===== SAMPLE RESPONSE =====\n")
    print(agent.get_response(example_queries[0]))
    print("\n==========================\n")

if __name__ == "__main__":
    main()


===== SAMPLE RESPONSE =====

**Personalized Job Recommendations**

Based on your profile and query, I've curated five job opportunities that align with your skills, experience, and preferences. Here are the recommendations:

**Job 1: AI/ML Engineer**
**Company:** HCL Technologies
**Location:** Bangalore (open to relocation)

This job is a great match for your skills in AI, ML, and Python. As an AI/ML Engineer, you'll work on developing and deploying AI models for various industries. With your experience as an AI developer at OLVT, you'll be able to leverage your knowledge to contribute to HCL's innovative projects.

[Apply on HCL Technologies' Career Page](https://www.hcltech.com/careers/job-search)

**Job 2: Software Engineer - AI/ML**
**Company:** Accenture
**Location:** Hyderabad (open to relocation)

This role requires expertise in AI, ML, and Python, which aligns perfectly with your skills. As a Software Engineer, you'll work on developing AI-powered solutions for clients across 

In [3]:
import os
import json
import requests
from datetime import datetime
from dotenv import load_dotenv
from groq import Groq

# Load environment variables
load_dotenv()

# Initialize Groq client
groq_api_key = os.getenv("GROQ_API_KEY")
client = Groq(api_key=groq_api_key)

class JobRecommendationAgent:
    def __init__(self):
        self.job_sources = {
            'linkedin': 'https://www.linkedin.com/jobs/',
            'internshala': 'https://internshala.com/',
            'naukri': 'https://www.naukri.com/',
            'indeed': 'https://www.indeed.co.in/'
        }
        self.community_sources = {
            'linkedin_groups': 'https://www.linkedin.com/groups/',
            'meetup': 'https://www.meetup.com/',
            'discord': 'https://discord.com/servers'
        }
        self.event_sources = {
            'linkedin_events': 'https://www.linkedin.com/events/',
            'internshala_trainings': 'https://internshala.com/trainings/',
            'meetup_events': 'https://www.meetup.com/find/events/'
        }
        
    def fetch_user_profile(self, user_id):
        """Fetch user profile from MongoDB (this would be implemented in production)"""
        # In a real implementation, this would query MongoDB
        # For the purpose of this prototype, we'll use the sample data
        
        sample_profile = {
            "_id": {"$oid": "6809c3daa03a7a1e240ab91f"},
            "user_id": "6809c002a03a7a1e240ab91e",
            "education": "Bachelor's Degree",
            "skills": ["Python", "Java", "AI", "NLP", "ML", "DL", "web development", "app development"],
            "current_status": "Looking for Work",
            "experience_years": {"$numberInt": "1"},
            "last_job": {"title": "AI developer", "company": "OLVT"},
            "life_stage": {"pregnancy_status": "No", "needs_flexible_work": False, "situation": "None of the above"},
            "job_preferences": {
                "type": "Remote Work",
                "roles": ["Software"],
                "short_term_goal": "Upskill and crack good placement",
                "long_term_goal": "Yes, i want to be an entrepreneur"
            },
            "location": {"city": "Tirupati", "relocation": True, "work_mode": "Flexible"},
            "community": {"wants_mentorship": True, "mentorship_type": "Skill development", "join_events": True},
            "communication_preference": "Email",
            "consent": True,
            "created_at": {"$date": {"$numberLong": "1745470426051"}}
        }
        
        return sample_profile
    
    def normalize_profile(self, profile):
        """Normalize the profile data for easier use"""
        normalized = {
            "user_id": profile.get("user_id", ""),
            "education": profile.get("education", ""),
            "skills": profile.get("skills", []),
            "current_status": profile.get("current_status", ""),
            "experience_years": int(profile.get("experience_years", {}).get("$numberInt", 0)),
            "last_job_title": profile.get("last_job", {}).get("title", ""),
            "last_job_company": profile.get("last_job", {}).get("company", ""),
            "job_preferences": {
                "type": profile.get("job_preferences", {}).get("type", ""),
                "roles": profile.get("job_preferences", {}).get("roles", []),
                "short_term_goal": profile.get("job_preferences", {}).get("short_term_goal", ""),
                "long_term_goal": profile.get("job_preferences", {}).get("long_term_goal", "")
            },
            "location": {
                "city": profile.get("location", {}).get("city", ""),
                "relocation": profile.get("location", {}).get("relocation", False),
                "work_mode": profile.get("location", {}).get("work_mode", "")
            },
            "community_interests": {
                "wants_mentorship": profile.get("community", {}).get("wants_mentorship", False),
                "mentorship_type": profile.get("community", {}).get("mentorship_type", ""),
                "join_events": profile.get("community", {}).get("join_events", False)
            }
        }
        return normalized
    
    def create_profile_summary(self, profile):
        """Create a summary of the profile for the LLM"""
        normalized = self.normalize_profile(profile)
        
        summary = f"""
        User Profile Summary:
        - Education: {normalized['education']}
        - Skills: {', '.join(normalized['skills'])}
        - Current Status: {normalized['current_status']}
        - Experience: {normalized['experience_years']} year(s)
        - Last Job: {normalized['last_job_title']} at {normalized['last_job_company']}
        - Job Preferences: 
            - Type: {normalized['job_preferences']['type']}
            - Roles: {', '.join(normalized['job_preferences']['roles'])}
            - Location: {normalized['location']['city']}
            - Willing to relocate: {normalized['location']['relocation']}
            - Preferred work mode: {normalized['location']['work_mode']}
        - Community Interests:
            - Interested in mentorship: {normalized['community_interests']['wants_mentorship']}
            - Mentorship type: {normalized['community_interests']['mentorship_type']}
            - Interested in events: {normalized['community_interests']['join_events']}
        """
        return summary
    
    def get_recommendation(self, user_id, query_type, query_details=""):
        """
        Get recommendation using Groq API
        query_type: 'jobs', 'communities', 'events'
        query_details: any specific requirements from the user
        """
        profile = self.fetch_user_profile(user_id)
        profile_summary = self.create_profile_summary(profile)
        
        current_date = datetime.now().strftime("%Y-%m-%d")
        
        if query_type == "jobs":
            prompt = f"""
            Today's date: {current_date}

            I need you to act as a job recommendation agent. Based on the user profile below, suggest 5 specific, relevant job opportunities that match their skills, experience, and preferences. Include the job title, company, location, whether it's remote/hybrid/onsite, and a direct URL to apply.

            {profile_summary}

            Additional requirements: {query_details}

            For each job recommendation:
            1. Provide actual, current job listings from platforms like LinkedIn, Internshala, Naukri.com, or company career pages
            2. Include specific job titles and company names
            3. Explain why this job matches the user's profile in 1-2 sentences
            4. Include a working URL for the job application
            5. Focus on recent postings (within the last 2 weeks if possible)

            Present your recommendations in a structured format with useful details.
            """
            
            sources = self.job_sources
            
        elif query_type == "communities":
            prompt = f"""
            Today's date: {current_date}

            I need you to act as a community recommendation agent. Based on the user profile below, suggest 5 specific, relevant professional communities, groups, or forums that match their skills, interests, and career goals. Include the group name, platform, focus area, and a direct URL to join.

            {profile_summary}

            Additional requirements: {query_details}

            For each community recommendation:
            1. Provide actual, existing communities on platforms like LinkedIn Groups, Discord servers, Slack workspaces, or other professional networking platforms
            2. Include specific group names
            3. Explain why this community matches the user's profile and how it can benefit them in 1-2 sentences
            4. Include a working URL to join or learn more about the community
            5. Focus on active communities with engaged members

            Present your recommendations in a structured format with useful details.
            """
            
            sources = self.community_sources
            
        elif query_type == "events":
            prompt = f"""
            Today's date: {current_date}

            I need you to act as an event recommendation agent. Based on the user profile below, suggest 5 specific, relevant upcoming events, webinars, or workshops that match their skills, interests, and career goals. Include the event name, organizer, date, format (virtual/in-person), and a direct URL to register.

            {profile_summary}

            Additional requirements: {query_details}

            For each event recommendation:
            1. Provide actual, upcoming events from platforms like LinkedIn Events, Internshala, university pages, or tech community websites
            2. Include specific event names, organizers, and dates
            3. Explain why this event matches the user's profile and how it can benefit them in 1-2 sentences
            4. Include a working URL to register for the event
            5. Focus on events happening within the next 4 weeks

            Present your recommendations in a structured format with useful details.
            """
            
            sources = self.event_sources
            
        else:
            return {"error": "Invalid query type. Must be 'jobs', 'communities', or 'events'."}
        
        # Add source information to the prompt
        prompt += f"\n\nRelevant sources to consider: {', '.join(sources.keys())}"
        
        # Call Groq API
        response = client.chat.completions.create(
            model="llama3-70b-8192",  # Using Llama3 70B model via Groq
            messages=[
                {"role": "system", "content": "You are an AI career advisor helping users find relevant jobs, communities, and events based on their profile. Provide specific, actionable recommendations with up-to-date information and working links."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=2048,
            temperature=0.7
        )
        
        return {
            "query_type": query_type,
            "recommendations": response.choices[0].message.content,
            "timestamp": datetime.now().isoformat()
        }

    def process_user_query(self, user_id, query):
        """Process natural language query from user and route to appropriate recommendation type"""
        # Use Groq to classify the query type
        classification_prompt = f"""
        The following is a user query about career recommendations. Classify this query into one of these categories:
        - jobs (if asking about job opportunities, positions, openings, career options)
        - communities (if asking about professional groups, communities, forums, networks)
        - events (if asking about webinars, workshops, conferences, meetups, training sessions)
        
        Additionally, extract any specific requirements or preferences mentioned in the query.
        
        User query: "{query}"
        
        Respond in JSON format:
        {{
            "category": "jobs|communities|events",
            "details": "extracted requirements or preferences"
        }}
        """
        
        classification_response = client.chat.completions.create(
            model="llama3-70b-8192",
            messages=[
                {"role": "system", "content": "You analyze user queries and classify them into recommendation categories. Respond only with the requested JSON format."},
                {"role": "user", "content": classification_prompt}
            ],
            max_tokens=200,
            temperature=0.1
        )
        
        try:
            result = json.loads(classification_response.choices[0].message.content)
            category = result.get("category", "jobs")  # Default to jobs if classification fails
            details = result.get("details", "")
            
            return self.get_recommendation(user_id, category, details)
        except json.JSONDecodeError:
            # Fallback in case of parsing error
            return self.get_recommendation(user_id, "jobs", query)

# Example usage
if __name__ == "__main__":
    agent = JobRecommendationAgent()
    
    # Example queries
    user_id = "6809c002a03a7a1e240ab91e"
    
    # Test job recommendations
    job_query = "I'm looking for remote AI developer positions that match my skills"
    job_response = agent.process_user_query(user_id, job_query)
    print("\nJOB RECOMMENDATIONS:")
    print(job_response["recommendations"])
    
    # Test community recommendations
    community_query = "Suggest some tech communities where I can improve my machine learning skills"
    community_response = agent.process_user_query(user_id, community_query)
    print("\nCOMMUNITY RECOMMENDATIONS:")
    print(community_response["recommendations"])
    
    # Test event recommendations
    event_query = "Are there any upcoming AI or ML webinars I can attend to improve my skills?"
    event_response = agent.process_user_query(user_id, event_query)
    print("\nEVENT RECOMMENDATIONS:")
    print(event_response["recommendations"])


JOB RECOMMENDATIONS:
Based on the user profile, I've curated five job recommendations that match their skills, experience, and preferences. Here are the recommendations:

**Job Recommendation 1:**
**Job Title:** AI/ML Engineer
**Company:** HCL Technologies
**Location:** Bangalore (willing to relocate)
**Work Mode:** Remote
**URL:** https://www.naukri.com/job-listings-ai-ml-engineer-hcl-technologies-bangalore-jobid-220417002333

This job matches the user's profile as it requires skills in AI, ML, and Python, and offers a remote work arrangement. HCL Technologies is a well-established company in the industry, and this role aligns with the user's experience as an AI developer.

**Job Recommendation 2:**
**Job Title:** Senior Software Engineer - AI/ML
**Company:** Accenture
**Location:** Chennai (willing to relocate)
**Work Mode:** Hybrid
**URL:** https://www.linkedin.com/jobs/view/senior-software-engineer-ai-ml-at-accenture-3307412333

This job aligns with the user's skills in AI, ML, an

In [4]:
import os
import json
import requests
import re
from datetime import datetime
from dotenv import load_dotenv
from groq import Groq
from bs4 import BeautifulSoup
from serpapi import GoogleSearch
import time
import random
from urllib.parse import quote_plus
from concurrent.futures import ThreadPoolExecutor

# Load environment variables
load_dotenv()

# Initialize Groq client
groq_api_key = os.getenv("GROQ_API_KEY")
client = Groq(api_key=groq_api_key)

# Initialize SerpAPI client for web search
serpapi_key = os.getenv("SERPAPI_KEY")

class WebTools:
    """Tools for web scraping and search"""
    
    @staticmethod
    def search_web(query, num_results=5):
        """Search the web using SerpAPI"""
        try:
            search_params = {
                "engine": "google",
                "q": query,
                "api_key": serpapi_key,
                "num": num_results,
                "gl": "in"  # Focus on India results
            }
            search = GoogleSearch(search_params)
            results = search.get_dict()
            
            if "organic_results" not in results:
                return []
                
            return [
                {
                    "title": result.get("title", ""),
                    "link": result.get("link", ""),
                    "snippet": result.get("snippet", "")
                }
                for result in results["organic_results"][:num_results]
            ]
        except Exception as e:
            print(f"Search error: {e}")
            return []
    
    @staticmethod
    def scrape_webpage(url):
        """Scrape content from a webpage"""
        try:
            headers = {
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
            }
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()
            
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Remove script and style elements
            for script_or_style in soup(['script', 'style']):
                script_or_style.decompose()
                
            # Extract text content
            text = soup.get_text(separator=' ', strip=True)
            
            # Clean up whitespace
            lines = (line.strip() for line in text.splitlines())
            chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
            text = '\n'.join(chunk for chunk in chunks if chunk)
            
            # Limit to reasonable size
            return text[:10000]
        except Exception as e:
            print(f"Scraping error for {url}: {e}")
            return ""

class JobScraper:
    """Specialized scraper for job platforms"""
    
    @staticmethod
    def scrape_linkedin_jobs(search_term):
        """Scrape LinkedIn jobs based on search term"""
        encoded_search = quote_plus(search_term)
        url = f"https://www.linkedin.com/jobs/search/?keywords={encoded_search}"
        
        try:
            headers = {
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
            }
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()
            
            soup = BeautifulSoup(response.text, 'html.parser')
            
            jobs = []
            job_cards = soup.select('.job-search-card')
            
            for card in job_cards[:5]:  # Limit to 5 results
                job_link_element = card.select_one('.job-card-container__link')
                job_link = job_link_element['href'] if job_link_element else ''
                
                job_title_element = card.select_one('.job-card-list__title')
                job_title = job_title_element.text.strip() if job_title_element else ''
                
                company_element = card.select_one('.job-card-container__company-name')
                company = company_element.text.strip() if company_element else ''
                
                location_element = card.select_one('.job-card-container__metadata-item')
                location = location_element.text.strip() if location_element else ''
                
                jobs.append({
                    'title': job_title,
                    'company': company,
                    'location': location,
                    'link': job_link if job_link.startswith('http') else f"https://www.linkedin.com{job_link}"
                })
                
            return jobs
        except Exception as e:
            print(f"LinkedIn scraping error: {e}")
            return []
    
    @staticmethod
    def scrape_internshala(search_term):
        """Scrape Internshala jobs/internships based on search term"""
        encoded_search = quote_plus(search_term)
        url = f"https://internshala.com/internships/{encoded_search}"
        
        try:
            headers = {
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
            }
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()
            
            soup = BeautifulSoup(response.text, 'html.parser')
            
            jobs = []
            job_containers = soup.select('.internship_meta')
            
            for container in job_containers[:5]:  # Limit to 5 results
                title_element = container.select_one('.view_detail_button')
                title = title_element.text.strip() if title_element else ''
                
                company_element = container.select_one('.company_name')
                company = company_element.text.strip() if company_element else ''
                
                location_element = container.select_one('.location_link')
                location = location_element.text.strip() if location_element else ''
                
                link_element = container.select_one('a.view_detail_button')
                link = link_element['href'] if link_element and 'href' in link_element.attrs else ''
                
                jobs.append({
                    'title': title,
                    'company': company,
                    'location': location,
                    'link': link if link.startswith('http') else f"https://internshala.com{link}"
                })
                
            return jobs
        except Exception as e:
            print(f"Internshala scraping error: {e}")
            return []
    
    @staticmethod
    def scrape_naukri(search_term):
        """Scrape Naukri.com jobs based on search term"""
        encoded_search = quote_plus(search_term)
        url = f"https://www.naukri.com/jobs-in-india-{encoded_search}"
        
        try:
            headers = {
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
            }
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()
            
            soup = BeautifulSoup(response.text, 'html.parser')
            
            jobs = []
            job_articles = soup.select('article.jobTuple')
            
            for article in job_articles[:5]:  # Limit to 5 results
                title_element = article.select_one('a.title')
                title = title_element.text.strip() if title_element else ''
                
                link = title_element['href'] if title_element and 'href' in title_element.attrs else ''
                
                company_element = article.select_one('a.subTitle')
                company = company_element.text.strip() if company_element else ''
                
                location_element = article.select_one('.locWdth')
                location = location_element.text.strip() if location_element else ''
                
                jobs.append({
                    'title': title,
                    'company': company,
                    'location': location,
                    'link': link
                })
                
            return jobs
        except Exception as e:
            print(f"Naukri scraping error: {e}")
            return []

class EventScraper:
    """Specialized scraper for events and webinars"""
    
    @staticmethod
    def scrape_meetup_events(search_term):
        """Scrape Meetup events based on search term"""
        encoded_search = quote_plus(search_term)
        url = f"https://www.meetup.com/find/?keywords={encoded_search}"
        
        try:
            headers = {
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
            }
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()
            
            soup = BeautifulSoup(response.text, 'html.parser')
            
            events = []
            event_cards = soup.select('.eventCard')
            
            for card in event_cards[:5]:  # Limit to 5 results
                title_element = card.select_one('.eventCardHead--title')
                title = title_element.text.strip() if title_element else ''
                
                link_element = card.select_one('a')
                link = link_element['href'] if link_element and 'href' in link_element.attrs else ''
                
                date_element = card.select_one('.eventTimeDisplay')
                date = date_element.text.strip() if date_element else ''
                
                organizer_element = card.select_one('.eventCardHead--groupName')
                organizer = organizer_element.text.strip() if organizer_element else ''
                
                events.append({
                    'title': title,
                    'organizer': organizer,
                    'date': date,
                    'link': link
                })
                
            return events
        except Exception as e:
            print(f"Meetup scraping error: {e}")
            return []
    
    @staticmethod
    def search_linkedin_events(search_term):
        """Search for LinkedIn events"""
        encoded_search = quote_plus(search_term)
        search_query = f"LinkedIn events {search_term}"
        
        try:
            search_results = WebTools.search_web(search_query, 5)
            events = []
            
            for result in search_results:
                if 'linkedin.com/events' in result['link']:
                    events.append({
                        'title': result['title'],
                        'description': result['snippet'],
                        'link': result['link']
                    })
            
            return events
        except Exception as e:
            print(f"LinkedIn events search error: {e}")
            return []

class CommunityFinder:
    """Tools to find relevant communities"""
    
    @staticmethod
    def find_linkedin_groups(search_term):
        """Find LinkedIn groups based on search term"""
        encoded_search = quote_plus(search_term)
        search_query = f"LinkedIn groups for {search_term} professionals"
        
        try:
            search_results = WebTools.search_web(search_query, 5)
            groups = []
            
            for result in search_results:
                if 'linkedin.com/groups' in result['link']:
                    groups.append({
                        'name': result['title'],
                        'description': result['snippet'],
                        'link': result['link']
                    })
            
            return groups
        except Exception as e:
            print(f"LinkedIn groups search error: {e}")
            return []
    
    @staticmethod
    def find_discord_communities(search_term):
        """Find Discord communities based on search term"""
        search_query = f"Discord server community for {search_term} professionals"
        
        try:
            search_results = WebTools.search_web(search_query, 5)
            communities = []
            
            for result in search_results:
                if 'discord' in result['link'].lower():
                    communities.append({
                        'name': result['title'],
                        'description': result['snippet'],
                        'link': result['link']
                    })
            
            return communities
        except Exception as e:
            print(f"Discord communities search error: {e}")
            return []

class AIJobAgent:
    """Main agent class that coordinates recommendations using Groq API"""
    
    def __init__(self):
        self.web_tools = WebTools()
        self.job_scraper = JobScraper()
        self.event_scraper = EventScraper()
        self.community_finder = CommunityFinder()
    
    def fetch_user_profile(self, user_id):
        """Fetch user profile from MongoDB (this would be implemented in production)"""
        # In a real implementation, this would query MongoDB
        # For the purpose of this prototype, we'll use the sample data
        
        sample_profile = {
            "_id": {"$oid": "6809c3daa03a7a1e240ab91f"},
            "user_id": "6809c002a03a7a1e240ab91e",
            "education": "Bachelor's Degree",
            "skills": ["Python", "Java", "AI", "NLP", "ML", "DL", "web development", "app development"],
            "current_status": "Looking for Work",
            "experience_years": {"$numberInt": "1"},
            "last_job": {"title": "AI developer", "company": "OLVT"},
            "life_stage": {"pregnancy_status": "No", "needs_flexible_work": False, "situation": "None of the above"},
            "job_preferences": {
                "type": "Remote Work",
                "roles": ["Software"],
                "short_term_goal": "Upskill and crack good placement",
                "long_term_goal": "Yes, i want to be an entrepreneur"
            },
            "location": {"city": "Tirupati", "relocation": True, "work_mode": "Flexible"},
            "community": {"wants_mentorship": True, "mentorship_type": "Skill development", "join_events": True},
            "communication_preference": "Email",
            "consent": True,
            "created_at": {"$date": {"$numberLong": "1745470426051"}}
        }
        
        return sample_profile
    
    def normalize_profile(self, profile):
        """Normalize the profile data for easier use"""
        normalized = {
            "user_id": profile.get("user_id", ""),
            "education": profile.get("education", ""),
            "skills": profile.get("skills", []),
            "current_status": profile.get("current_status", ""),
            "experience_years": int(profile.get("experience_years", {}).get("$numberInt", 0)),
            "last_job_title": profile.get("last_job", {}).get("title", ""),
            "last_job_company": profile.get("last_job", {}).get("company", ""),
            "job_preferences": {
                "type": profile.get("job_preferences", {}).get("type", ""),
                "roles": profile.get("job_preferences", {}).get("roles", []),
                "short_term_goal": profile.get("job_preferences", {}).get("short_term_goal", ""),
                "long_term_goal": profile.get("job_preferences", {}).get("long_term_goal", "")
            },
            "location": {
                "city": profile.get("location", {}).get("city", ""),
                "relocation": profile.get("location", {}).get("relocation", False),
                "work_mode": profile.get("location", {}).get("work_mode", "")
            },
            "community_interests": {
                "wants_mentorship": profile.get("community", {}).get("wants_mentorship", False),
                "mentorship_type": profile.get("community", {}).get("mentorship_type", ""),
                "join_events": profile.get("community", {}).get("join_events", False)
            }
        }
        return normalized
    
    def create_profile_summary(self, profile):
        """Create a summary of the profile for the LLM"""
        normalized = self.normalize_profile(profile)
        
        summary = f"""
        User Profile Summary:
        - Education: {normalized['education']}
        - Skills: {', '.join(normalized['skills'])}
        - Current Status: {normalized['current_status']}
        - Experience: {normalized['experience_years']} year(s)
        - Last Job: {normalized['last_job_title']} at {normalized['last_job_company']}
        - Job Preferences: 
            - Type: {normalized['job_preferences']['type']}
            - Roles: {', '.join(normalized['job_preferences']['roles'])}
            - Location: {normalized['location']['city']}
            - Willing to relocate: {normalized['location']['relocation']}
            - Preferred work mode: {normalized['location']['work_mode']}
        - Community Interests:
            - Interested in mentorship: {normalized['community_interests']['wants_mentorship']}
            - Mentorship type: {normalized['community_interests']['mentorship_type']}
            - Interested in events: {normalized['community_interests']['join_events']}
        """
        return summary
    
    def generate_search_queries(self, user_profile, query_type, query_details=""):
        """Generate search queries based on user profile and query type"""
        normalized = self.normalize_profile(user_profile)
        skills_str = ", ".join(normalized["skills"])
        
        prompt = f"""
        Based on the following user profile and query type, generate 3 specific search queries that would help find the most relevant results. Each query should be tailored to search engines or job platforms.

        User profile:
        - Skills: {skills_str}
        - Experience: {normalized['experience_years']} year(s)
        - Last job: {normalized['last_job_title']}
        - Preferred work type: {normalized['job_preferences']['type']}
        - Location: {normalized['location']['city']}
        
        Query type: {query_type}
        Additional details from user: {query_details}
        
        Respond with exactly 3 search queries in a JSON array format like this:
        ["query1", "query2", "query3"]
        
        Make the queries specific and tailored to the user's background.
        """
        
        response = client.chat.completions.create(
            model="llama3-70b-8192",
            messages=[
                {"role": "system", "content": "You create specific search queries based on user profiles. Respond exactly in the format requested."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=300,
            temperature=0.7
        )
        
        try:
            queries = json.loads(response.choices[0].message.content)
            return queries
        except json.JSONDecodeError:
            # Fallback in case of parsing error
            content = response.choices[0].message.content
            # Try to extract array using regex
            match = re.search(r'\[(.*?)\]', content, re.DOTALL)
            if match:
                try:
                    queries = json.loads('[' + match.group(1) + ']')
                    return queries
                except:
                    pass
            
            # Last resort
            if query_type == "jobs":
                return [
                    f"{normalized['last_job_title']} jobs {normalized['location']['city']}",
                    f"{skills_str} {normalized['job_preferences']['type']} jobs",
                    f"entry level {normalized['last_job_title']} positions"
                ]
            elif query_type == "communities":
                return [
                    f"{skills_str} professional groups",
                    f"{normalized['last_job_title']} communities",
                    f"tech communities for {normalized['job_preferences']['short_term_goal']}"
                ]
            else:  # events
                return [
                    f"{skills_str} upcoming webinars",
                    f"{normalized['last_job_title']} workshops",
                    f"tech events in {normalized['location']['city']}"
                ]
    
    def fetch_jobs(self, search_queries):
        """Fetch jobs from multiple platforms using generated search queries"""
        all_jobs = []
        
        with ThreadPoolExecutor(max_workers=3) as executor:
            # Run LinkedIn searches
            linkedin_futures = [executor.submit(self.job_scraper.scrape_linkedin_jobs, query) for query in search_queries]
            # Run Internshala searches
            internshala_futures = [executor.submit(self.job_scraper.scrape_internshala, query) for query in search_queries]
            # Run Naukri searches
            naukri_futures = [executor.submit(self.job_scraper.scrape_naukri, query) for query in search_queries]
            
            # Collect LinkedIn results
            for future in linkedin_futures:
                try:
                    results = future.result()
                    for job in results:
                        job['source'] = 'LinkedIn'
                        all_jobs.append(job)
                except Exception as e:
                    print(f"Error in LinkedIn job search: {e}")
            
            # Collect Internshala results
            for future in internshala_futures:
                try:
                    results = future.result()
                    for job in results:
                        job['source'] = 'Internshala'
                        all_jobs.append(job)
                except Exception as e:
                    print(f"Error in Internshala job search: {e}")
            
            # Collect Naukri results
            for future in naukri_futures:
                try:
                    results = future.result()
                    for job in results:
                        job['source'] = 'Naukri'
                        all_jobs.append(job)
                except Exception as e:
                    print(f"Error in Naukri job search: {e}")
        
        # Remove duplicates
        unique_jobs = []
        seen_links = set()
        
        for job in all_jobs:
            link = job.get('link', '')
            if link and link not in seen_links:
                seen_links.add(link)
                unique_jobs.append(job)
        
        return unique_jobs[:10]  # Return top 10 unique jobs
    
    def fetch_communities(self, search_queries):
        """Fetch communities based on search queries"""
        all_communities = []
        
        with ThreadPoolExecutor(max_workers=2) as executor:
            # Run LinkedIn group searches
            linkedin_futures = [executor.submit(self.community_finder.find_linkedin_groups, query) for query in search_queries]
            # Run Discord community searches
            discord_futures = [executor.submit(self.community_finder.find_discord_communities, query) for query in search_queries]
            
            # Collect LinkedIn results
            for future in linkedin_futures:
                try:
                    results = future.result()
                    for community in results:
                        community['source'] = 'LinkedIn'
                        all_communities.append(community)
                except Exception as e:
                    print(f"Error in LinkedIn community search: {e}")
            
            # Collect Discord results
            for future in discord_futures:
                try:
                    results = future.result()
                    for community in results:
                        community['source'] = 'Discord'
                        all_communities.append(community)
                except Exception as e:
                    print(f"Error in Discord community search: {e}")
        
        # Remove duplicates
        unique_communities = []
        seen_links = set()
        
        for community in all_communities:
            link = community.get('link', '')
            if link and link not in seen_links:
                seen_links.add(link)
                unique_communities.append(community)
        
        return unique_communities[:7]  # Return top 7 unique communities
    
    def fetch_events(self, search_queries):
        """Fetch events based on search queries"""
        all_events = []
        
        with ThreadPoolExecutor(max_workers=2) as executor:
            # Run Meetup searches
            meetup_futures = [executor.submit(self.event_scraper.scrape_meetup_events, query) for query in search_queries]
            # Run LinkedIn event searches
            linkedin_futures = [executor.submit(self.event_scraper.search_linkedin_events, query) for query in search_queries]
            
            # Collect Meetup results
            for future in meetup_futures:
                try:
                    results = future.result()
                    for event in results:
                        event['source'] = 'Meetup'
                        all_events.append(event)
                except Exception as e:
                    print(f"Error in Meetup event search: {e}")
            
            # Collect LinkedIn results
            for future in linkedin_futures:
                try:
                    results = future.result()
                    for event in results:
                        event['source'] = 'LinkedIn'
                        all_events.append(event)
                except Exception as e:
                    print(f"Error in LinkedIn event search: {e}")
        
        # Remove duplicates
        unique_events = []
        seen_links = set()
        
        for event in all_events:
            link = event.get('link', '')
            if link and link not in seen_links:
                seen_links.add(link)
                unique_events.append(event)
        
        return unique_events[:7]  # Return top 7 unique events
    
    def rank_and_explain_recommendations(self, items, user_profile, item_type):
        """Use Groq to rank and explain why recommendations are relevant to the user"""
        if not items:
            return []
            
        normalized_profile = self.normalize_profile(user_profile)
        profile_summary = self.create_profile_summary(user_profile)
        
        # Convert items to a string representation
        items_str = json.dumps(items, indent=2)
        
        prompt = f"""
        I have a user with the following profile:
        
        {profile_summary}
        
        I've found these {item_type} that might be relevant:
        
        {items_str}
        
        For each {item_type}, I need you to:
        1. Rank how relevant it is to the user on a scale of 1-10
        2. Explain in 1-2 sentences why it's relevant to this specific user
        3. Return the results in JSON format
        
        Respond with a JSON array with each item containing:
        - The original item data
        - A "relevance_score" field (1-10)
        - A "relevance_explanation" field (1-2 sentences)
        
        Sort the results by relevance_score in descending order.
        """
        
        try:
            response = client.chat.completions.create(
                model="llama3-70b-8192",
                messages=[
                    {"role": "system", "content": "You analyze and explain why recommendations are relevant to a specific user profile. Respond in the exact JSON format requested."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=2000,
                temperature=0.3
            )
            
            try:
                ranked_items = json.loads(response.choices[0].message.content)
                return ranked_items
            except json.JSONDecodeError:
                # If we can't parse JSON, return the original items
                print("Failed to parse ranking response as JSON")
                for item in items:
                    item['relevance_score'] = 5
                    item['relevance_explanation'] = "This may be relevant based on your profile."
                return items
                
        except Exception as e:
            print(f"Error ranking items: {e}")
            # Add default relevance info
            for item in items:
                item['relevance_score'] = 5
                item['relevance_explanation'] = "This may be relevant based on your profile."
            return items
    
    def format_recommendations(self, ranked_items, item_type, user_profile):
        """Format recommendations into a readable response"""
        if not ranked_items:
            return f"I couldn't find any relevant {item_type} matching your profile. Try adjusting your search or preferences."
        
        normalized_profile = self.normalize_profile(user_profile)
        
        prompt = f"""
        I have a user with the following skills and preferences:
        - Skills: {', '.join(normalized_profile['skills'])}
        - Experience: {normalized_profile['experience_years']} year(s)
        - Last job: {normalized_profile['last_job_title']}
        - Job preferences: {normalized_profile['job_preferences']['type']}
        - Location: {normalized_profile['location']['city']}
        
        I've found and ranked these {item_type} for them:
        
        {json.dumps(ranked_items[:5], indent=2)}
        
        Please format these recommendations into a helpful, easy-to-read response that:
        1. Starts with a brief personalized introduction
        2. Lists the top 5 recommendations with their details and why they're relevant
        3. Each recommendation should include all available details and a clickable link
        4. Use markdown formatting to make it readable
        5. End with a brief personalized conclusion
        
        The tone should be professional but friendly.
        """
        
        response = client.chat.completions.create(
            model="llama3-70b-8192",
            messages=[
                {"role": "system", "content": "You craft personalized recommendation messages with clear formatting and helpful descriptions."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=1500,
            temperature=0.7
        )
        
        return response.choices[0].message.content
    
    def get_recommendations(self, user_id, query_type, query_details=""):
        """Get recommendations based on query type"""
        # Fetch and normalize user profile
        user_profile = self.fetch_user_profile(user_id)
        
        # Generate search queries based on profile and query type
        search_queries = self.generate_search_queries(user_profile, query_type, query_details)
        
        if query_type == "jobs":
            # Fetch jobs from multiple sources
            print(f"Searching for jobs with queries: {search_queries}")
            jobs = self.fetch_jobs(search_queries)
            
            # Rank and explain relevance
            ranked_jobs = self.rank_and_explain_recommendations(jobs, user_profile, "jobs")
            
            # Format response
            response = self.format_recommendations(ranked_jobs, "jobs", user_profile)
            
        elif query_type == "communities":
            # Fetch communities
            print(f"Searching for communities with queries: {search_queries}")
            communities = self.fetch_communities(search_queries)
            
            # Rank and explain relevance
            ranked_communities = self.rank_and_explain_recommendations(communities, user_profile, "communities")
            
            # Format response
            response = self.format_recommendations(ranked_communities, "communities", user_profile)
            
        elif query_type == "events":
            # Fetch events
            print(f"Searching for events with queries: {search_queries}")
            events = self.fetch_events(search_queries)
            
            # Rank and explain relevance
            ranked_events = self.rank_and_explain_recommendations(events, user_profile, "events")
            
            # Format response
            response = self.format_recommendations(ranked_events, "events", user_profile)
            
        else:
            response = "Invalid query type. Please specify 'jobs', 'communities', or 'events'."
        
        return {
            "query_type": query_type,
            "recommendations": response,
            "timestamp": datetime.now().isoformat()
        }
    
    def process_user_query(self, user_id, query):
        """Process natural language query from user and route to appropriate recommendation type"""
        # Use Groq to classify the query type
        classification_prompt = f"""
        The following is a user query about career recommendations. Classify this query into one of these categories:
        - jobs (if asking about job opportunities, positions, openings, career options)
        - communities (if asking about professional groups, communities, forums, networks)
        - events (if asking about webinars, workshops, conferences, meetups, training sessions)
        
        Additionally, extract any specific requirements or preferences mentioned in the query.
        
        User query: "{query}"
        
        Respond in JSON format:
        {{
            "category": "jobs|communities|events",
            "details": "extracted requirements or preferences"
        }}
        """
        
        classification_response = client.chat.completions.create(
            model="llama3-70b-8192",
            messages=[
                {"role": "system", "content": "You analyze user queries and classify them into recommendation categories. Respond only with the requested JSON format."},
                {"role": "user", "content": classification_prompt}
            ],
            max_tokens=200,
            temperature=0.1
        )
        
        try:
            result = json.loads(classification_response.choices[0].message.content)
            category = result.get("category", "jobs")  # Default to jobs if classification fails
            details = result.get("details", "")
            
            return self.get_recommendations(user_id, category, details)
        except json.JSONDecodeError:
            # Fallback in case of parsing error
            return self.get_recommendations(user_id, "jobs", query)

class AgentAPI:
    """API wrapper for the agent"""
    
    def __init__(self):
        self.agent = AIJobAgent()
    
    def handle_request(self, request_data):
        """Handle API request"""
        try:
            user_id = request_data.get("user_id")
            query = request_data.get("query", "")
            
            if not user_id:
                return {"error": "Missing user_id in request"}
                
            if not query:
                return {"error": "Missing query in request"}
            
            result = self.agent.process_user_query(user_id, query)
            return result
            
        except Exception as e:
            return {"error": str(e)}

# Example usage
if __name__ == "__main__":
    agent_api = AgentAPI()
    
    # Test queries
    test_requests = [
        {
            "user_id": "6809c002a03a7a1e240ab91e",
            "query": "Find me remote AI developer positions that match my skills"
        },
        {
            "user_id": "6809c002a03a7a1e240ab91e",
            "query": "Suggest tech communities where I can improve my machine learning skills"
        },
        {
            "user_id": "6809c002a03a7a1e240ab91e",
            "query": "Are there any upcoming AI or ML webinars I can attend to improve my skills?"
        }
    ]
    
    for i, request in enumerate(test_requests):
        print(f"\n\n===== TEST REQUEST {i+1} =====")
        print(f"Query: {request['query']}")
        
        response = agent_api.handle_request(request)
        print("\nRESPONSE:")
        print(response.get("recommendations", "Error: " + response.get("error", "Unknown error")))
        
        # Add a slight delay between requests
        if i < len(test_requests) - 1:
            time.sleep(2)




===== TEST REQUEST 1 =====
Query: Find me remote AI developer positions that match my skills
Searching for jobs with queries: ['remote AI developer jobs python java tirupati', 'AI NLP ML DL developer remote work opportunities in India', 'python java web app development remote jobs for AI experts in Tirupati']
Failed to parse ranking response as JSON

RESPONSE:
Here is a formatted response:

**Personalized Job Recommendations for You!**

Hi there! Based on your skills and preferences, I've curated a list of job opportunities that might interest you. As an AI developer with 1 year of experience, you've got a strong foundation in Python, Java, AI, NLP, ML, and DL, as well as web and app development. You're looking for remote work opportunities, and I've got you covered!

**Top Recommendations for You:**
-------------------------------

### 1. **The Bangalore Editorial** ([Internshala](https://internshala.com))
**Company:** The Bangalore Editorial
**Location:** Not specified
**Relevance 

===== TEST REQUEST 1 =====
Query: Find me remote AI developer positions that match my skills
Searching for jobs with queries: ['remote AI developer jobs python java tirupati', 'AI NLP ML DL developer remote work opportunities in India', 'python java web app development remote jobs for AI experts in Tirupati']
Failed to parse ranking response as JSON

RESPONSE:
Here is a formatted response:

**Personalized Job Recommendations for You!**
=============================================

Hi there! Based on your skills and preferences, I've curated a list of job opportunities that might interest you. As an AI developer with 1 year of experience, you've got a strong foundation in Python, Java, AI, NLP, ML, and DL, as well as web and app development. You're looking for remote work opportunities, and I've got you covered!

**Top Recommendations for You:**
-------------------------------

### 1. **The Bangalore Editorial** ([Internshala](https://internshala.com))
**Company:** The Bangalore Editorial
**Location:** Not specified
**Relevance Score:** 5/10
**Why it's relevant:** This opportunity may be a good fit based on your profile.

Please note that only one job opportunity was found that matches your preferences. If you'd like to explore more options, I can certainly help you with that!

**Best of Luck with Your Job Search!**
=====================================

I hope you find these recommendations helpful in your job search. Remember to tailor your resume and cover letter to showcase your skills and experience. If you need any further assistance or have questions, feel free to ask!


===== TEST REQUEST 2 =====
Query: Suggest tech communities where I can improve my machine learning skills
Searching for communities with queries: ['site:meetup.com Tirupati machine learning communities', 'python machine learning communities online remote', '(AI OR NLP OR DL) communities for beginners online']

RESPONSE:
I couldn't find any relevant communities matching your profile. Try adjusting your search or preferences.


===== TEST REQUEST 3 =====
Query: Are there any upcoming AI or ML webinars I can attend to improve my skills?
Searching for events with queries: ['python AI ML online webinars for developers', '(AI or ML) and (web development or app development) online conferences near Tirupati or remote', 'upcoming NLP deep learning online meetups for junior developers']

RESPONSE:
I couldn't find any relevant events matching your profile. Try adjusting your search or preferences.